In [1]:
import numpy as np 
import pandas as pd
import pulp
import json

### Goals: 
###        Production cost for the original recipe
###        Production cost for recipe with fixed constituents
###        Production cost for variable constituents

In [151]:
df = pd.read_excel('ice_cream_recipe.xlsx', 
                skiprows=1, 
                sheet_name=['original_recipe','constituents','constituent_limits','costs'], 
                index_col=0)

#fixed variables
weight_value = 50
f_c = 1 #set to 1 for fixed value of constituents, 0 for variable constituents
v_c = 0 #set to 1 for variable constituents

In [148]:
num_ingredients = len(df['costs'].columns)
num_constituents = len(df['constituent_limits'].index)

print(num_ingredients, num_constituents)

14 7


In [252]:
df['constituents'].fillna(0,inplace=True)

In [272]:
df['constituent_limits']

,Minimum,Maximum,Requirement
Constituent,,,
Fat,0.150,0.170,0.1600
Serum Solids,0.070,0.090,0.0800
Sugar Solids,0.155,0.165,0.1600
Egg Solids,0.003,0.004,0.0035
Stabilizer,0.002,0.003,0.0025
Emulsifier,0.001,0.002,0.0015
Water,0.580,0.595,0.5925


In [205]:
#np.array(df['original_recipe']).reshape(7,0)


original_recipe_weight = np.array([df['original_recipe'].loc[ingredient][0] if ingredient in df['original_recipe'].index else 0 
                            for ingredient in df['costs'].columns])

(original_recipe_weight * df['constituents'].to_numpy()).sum()

49.9

In [138]:
df['total_cost'] = {'Original Recipe': sum([df['original_recipe'].loc[i]*df['costs'][i]['cost'] for i in df['original_recipe'].index])[0]}

In [141]:
original_recipe_cost = df['total_cost']['Original Recipe']
print(f'Total cost for original recipe is ${original_recipe_cost:,.2f} per 50 pound batch')


Total cost for original recipe is $28.60 per 50 pound batch


In [266]:
#Create the model
model = pulp.LpProblem(name='ice_cream_recipe', sense=pulp.LpMinimize)

#Create a matrix of possible ingredients and constituents
variable_name = [str(i) for i in range(1, num_ingredients+1)]

ingredient_variables = pulp.LpVariable.matrix('I', variable_name,
                                             lowBound=0, cat=pulp.LpContinuous)

model.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(ingredient_variables),
    sense=pulp.LpConstraintEQ,
    name='recipe_weight',
    rhs=weight_value))


for constituent in df['constituent_limits'].index:
    model.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum((ingredient_variables* df['constituents'].loc[constituent].values)/weight_value),
        sense=pulp.LpConstraintEQ,
        name='proportion_' + constituent,
        rhs=df['constituent_limits']['Requirement'].loc[constituent]))

#set objective
objective = pulp.lpSum(ingredient_variables * df['costs'].values)                               

model.setObjective(objective)

In [268]:
model.solve()

1

In [270]:
if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.2f}')
    output = []
    for i,var in enumerate(model.variables()):
        output.append(var.value())

else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $25.35


In [288]:
#Create the model
model2 = pulp.LpProblem(name='ice_cream_recipe', sense=pulp.LpMinimize)

#Create a matrix of possible ingredients and constituents
variable_name = [str(i) for i in range(1, num_ingredients+1)]

ingredient_variables = pulp.LpVariable.matrix('I', variable_name,
                                             lowBound=0, cat=pulp.LpContinuous)

model2.addConstraint(pulp.LpConstraint(
    e=pulp.lpSum(ingredient_variables),
    sense=pulp.LpConstraintEQ,
    name='recipe_weight',
    rhs=weight_value))


for constituent in df['constituent_limits'].index:
    model2.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum((ingredient_variables* df['constituents'].loc[constituent].values)/weight_value),
        sense=pulp.LpConstraintGE,
        name='low_proportion_' + constituent,
        rhs=df['constituent_limits']['Minimum'].loc[constituent]))

for constituent in df['constituent_limits'].index:
    model2.addConstraint(pulp.LpConstraint(
        e=pulp.lpSum((ingredient_variables* df['constituents'].loc[constituent].values)/weight_value),
        sense=pulp.LpConstraintLE,
        name='high_proportion_' + constituent,
        rhs=df['constituent_limits']['Maximum'].loc[constituent]))

#set objective
objective2 = pulp.lpSum(ingredient_variables * df['costs'].values)                               

model2.setObjective(objective2)

In [289]:
model2

ice_cream_recipe:
MINIMIZE
1.19*I_1 + 1.75*I_10 + 4.45*I_11 + 2.45*I_12 + 1.65*I_13 + 0.01*I_14 + 0.7*I_2 + 2.32*I_3 + 2.3*I_4 + 2.87*I_5 + 0.25*I_6 + 0.35*I_7 + 0.65*I_8 + 0.25*I_9 + 0.0
SUBJECT TO
recipe_weight: I_1 + I_10 + I_11 + I_12 + I_13 + I_14 + I_2 + I_3 + I_4 + I_5
 + I_6 + I_7 + I_8 + I_9 = 50

low_proportion_Fat: 0.008 I_1 + 0.01 I_10 + 0.01 I_11 + 0.004 I_2 + 0.016 I_3
 + 0.016 I_4 + 0.018 I_5 + 0.002 I_6 >= 0.15

low_proportion_Serum_Solids: 0.002 I_1 + 0.002 I_4 + 0.002 I_6 + 0.006 I_7
 + 0.02 I_8 >= 0.07

low_proportion_Sugar_Solids: 0.002 I_10 + 0.014 I_9 >= 0.155

low_proportion_Egg_Solids: 0.008 I_10 + 0.01 I_11 >= 0.003

low_proportion_Stabilizer: 0.02 I_12 >= 0.002

low_proportion_Emulsifier: 0.02 I_13 >= 0.001

low_proportion_Water: 0.01 I_1 + 0.02 I_14 + 0.016 I_2 + 0.004 I_3 + 0.002 I_4
 + 0.002 I_5 + 0.016 I_6 + 0.014 I_7 + 0.006 I_9 >= 0.58

high_proportion_Fat: 0.008 I_1 + 0.01 I_10 + 0.01 I_11 + 0.004 I_2 + 0.016 I_3
 + 0.016 I_4 + 0.018 I_5 + 0.002 I_6 <= 

In [290]:
model2.solve()

if model2.status == 1:
    print(f'status: {model2.status}, {pulp.LpStatus[model2.status]}')
    print(f'objective: ${model2.objective.value():,.2f}')
    output = []
    for i,var in enumerate(model2.variables()):
        output.append(var.value())

else:
    print(f'status: {model2.status}, {pulp.LpStatus[model2.status]}')

status: 1, Optimal
objective: $24.04
